##Install Packages

In [32]:
!pip install cmake 'gym[atari]' scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Importing Libraries

In [49]:
import gym 
from IPython.display import clear_output
from time import sleep
import random
from IPython.display import clear_output
import numpy as np

# Set up the environment

In [50]:
def agent_enviro (state):
  env = gym.make("Taxi-v3").env
  env.render()
  env.reset() # reset environment to a new, random state
  env.s = state
  env.render()
  print("Action Space {}".format(env.action_space))
  print("State Space {}".format(env.observation_space))
  return state,env

In [51]:
state,env = agent_enviro(414)
state

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


414

# Q-learning approach

In [66]:
def Train_the_agent(game, alpha,gamma,epsilon):
  %%time
  """Training the agent"""
  env = gym.make(game).env
  env.reset()
  env.render()
  decay_factor=1e-3

  # Initialize the q table
  q_table = np.zeros([env.observation_space.n, env.action_space.n])
  # Initialize the q table
  q_table = np.zeros([env.observation_space.n, env.action_space.n])

  # Hyperparameters
  #alpha = 0.1
  #gamma = 0.6
  #epsilon = 0.1

  # For plotting metrics
  all_epochs = []
  all_penalties = []

  for i in range(1, 100001):
      state = env.reset()

      epochs, penalties, reward, = 0, 0, 0
      done = False
      if i % 10000==0:
        epsilon-=decay_factor
      
      while not done:
          if random.uniform(0, 1) < epsilon:
              action = env.action_space.sample() # Explore action space
          else:
              action = np.argmax(q_table[state]) # Exploit learned values

          next_state, reward, done, info = env.step(action) 
          
          old_value = q_table[state, action]
          next_max = np.max(q_table[next_state])
      
          new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
          q_table[state, action] = new_value

          if reward == -10:
              penalties += 1

          state = next_state
          epochs += 1
          
      if i % 100 == 0:
          clear_output(wait=True)
          print(f"Episode: {i}")

  print("Training finished.\n")
  return q_table

In [68]:
q_table = Train_the_agent("Taxi-v3",0.1,0.6,0.1)
print(q_table)

Episode: 100000
Training finished.

[[  0.           0.           0.           0.           0.
    0.        ]
 [ -2.41837056  -2.36395107  -2.41837034  -2.3639511   -2.27325184
  -11.3638796 ]
 [ -1.87014387  -1.45024021  -1.87014398  -1.45024012  -0.7504
  -10.45023915]
 ...
 [ -0.97647143   0.41599899  -1.02677684  -1.07474385  -3.44152954
   -3.44457388]
 [ -2.15651358  -2.12203237  -2.14510113  -2.12203606  -5.76585125
   -5.66999129]
 [  0.90524      1.09096448   2.19711119  11.          -1.98282408
   -1.98284877]]


# Evalutation

In [69]:
"""Evaluate agent's performance after Q-learning"""
def Evaluation(q_table):
  total_epochs, total_penalties = 0, 0
  episodes = 1000

  for _ in range(episodes):
      state = env.reset()
      epochs, penalties, reward = 0, 0, 0
      
      done = False
      
      while not done:
          action = np.argmax(q_table[state])
          state, reward, done, info = env.step(action)

          if reward == -10:
              penalties += 1

          epochs += 1

      total_penalties += penalties
      total_epochs += epochs

  print(f"Results after {episodes} episodes:")
  print(f"Average timesteps per episode: {total_epochs / episodes}")
  print(f"Average penalties per episode: {total_penalties / episodes}")

In [70]:
Evaluation(q_table)


Results after 1000 episodes:
Average timesteps per episode: 13.186
Average penalties per episode: 0.0
